In [1]:
import numpy as np
import pandas as pd
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt

In [2]:
nlp=spacy.load('en',disable=['parser','tagger','ner'])
def readfile(src):
    with open(src) as  rd:
        
        ret=rd.read()
    return ret 

In [3]:
generated=readfile('F:/pycharmprojects/Sir Allama Iqbal 2.txt')

In [4]:
def rmvpunc(src):
    tk= [token.text.lower() for token in nlp(src) if token.text not in [nlp.Defaults.stop_words]]
    return [token for token in tk if token not in '",.)\n\n":;...... ... (   -']

In [5]:
ntokens=rmvpunc(generated)

In [6]:
total_tokens=len(ntokens)
total_tokens

3234

In [7]:
trainlen=30+1


In [8]:
raw_sequence=[]
for i in range(trainlen,total_tokens):
    appnd=ntokens[i-trainlen:i]
    raw_sequence.append(appnd)

In [9]:
' '.join(raw_sequence[0])

'with the defeat of the sikhs in punjab by the british army western missionaries wasted no time in establishing centres of learning in sialkot one of these the scotch mission college'

In [10]:
' '.join(raw_sequence[1])

'the defeat of the sikhs in punjab by the british army western missionaries wasted no time in establishing centres of learning in sialkot one of these the scotch mission college founded'

In [11]:
tokenizer= Tokenizer()

In [12]:
tokenizer.fit_on_texts(raw_sequence)
seq = tokenizer.texts_to_sequences(raw_sequence)

In [13]:
for i in seq[0]:
    print(i," ",tokenizer.index_word[i])
    

46   with
1   the
390   defeat
2   of
1   the
1141   sikhs
3   in
387   punjab
16   by
1   the
71   british
1139   army
189   western
1136   missionaries
1135   wasted
93   no
187   time
3   in
1134   establishing
1133   centres
2   of
1132   learning
3   in
1130   sialkot
56   one
2   of
92   these
1   the
186   scotch
185   mission
191   college


In [14]:
vocabsize=len(tokenizer.word_counts)
vocabsize

1142

In [15]:
final_seq=np.array(seq)

In [16]:
final_seq

array([[  46,    1,  390, ...,  186,  185,  191],
       [   1,  390,    2, ...,  185,  191,  391],
       [ 390,    2,    1, ...,  191,  391,    3],
       ...,
       [1126,    2, 1127, ...,  389, 1142,    2],
       [   2, 1127,   23, ..., 1142,    2,    1],
       [1127,   23,    1, ...,    2,    1,  127]])

In [17]:
x=final_seq[:,:-1]
y=final_seq[:,-1]

x.shape

(3203, 30)

In [18]:
from tensorflow.keras.utils import to_categorical

In [19]:
y= to_categorical(y)
y.shape

(3203, 1143)

In [20]:
seqln=x.shape[1]

In [21]:
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping 
call = EarlyStopping(monitor='val_loss')

In [22]:
#call= EarlyStopping(monitor='loss',patience=2)
def createmodel(vsize,trainl,seqlen):
    model=Sequential()
    model.add(Embedding((vsize+1),trainl,input_length=seqlen))
    model.add(LSTM(200,return_sequences=True))
    model.add(LSTM(50))
    
    
    model.add(Dense(50,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(400 , activation='relu'))
    model.add(Dense(vsize+1,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])
    model.summary()
    
    return model

In [23]:
mymodel=createmodel(vocabsize,trainlen,seqln)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 31)            35433     
_________________________________________________________________
lstm (LSTM)                  (None, 30, 200)           185600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                50200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               40400     
_________________________________________________________________
dense_3 (Dense)              (None, 1143)              4

In [ ]:

mymodel.fit(x,y,epochs=100,verbose=1,validation_split=0.2,batch_size=128)

Train on 2562 samples, validate on 641 samples
Epoch 1/100
2562/2562 [==============================] - 16s 6ms/sample - loss: 6.6503 - accuracy: 0.0757 - val_loss: 6.3800 - val_accuracy: 0.0811
Epoch 2/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 5.9557 - accuracy: 0.0800 - val_loss: 6.6942 - val_accuracy: 0.0811
Epoch 3/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 5.8592 - accuracy: 0.0800 - val_loss: 6.9231 - val_accuracy: 0.0811
Epoch 4/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 5.8448 - accuracy: 0.0800 - val_loss: 6.9501 - val_accuracy: 0.0811
Epoch 5/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 5.8254 - accuracy: 0.0800 - val_loss: 7.0314 - val_accuracy: 0.0811
Epoch 6/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 5.8271 - accuracy: 0.0800 - val_loss: 7.0873 - val_accuracy: 0.0811
Epoch 7/100
2562/2562 [==============================] - 5s 2ms/sample

Epoch 56/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.8335 - accuracy: 0.1616 - val_loss: 14.3616 - val_accuracy: 0.0655
Epoch 57/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.7972 - accuracy: 0.1624 - val_loss: 14.5246 - val_accuracy: 0.0624
Epoch 58/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.8158 - accuracy: 0.1604 - val_loss: 14.3094 - val_accuracy: 0.0671
Epoch 59/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.7810 - accuracy: 0.1593 - val_loss: 14.6113 - val_accuracy: 0.0718
Epoch 60/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.7169 - accuracy: 0.1678 - val_loss: 14.7268 - val_accuracy: 0.0655
Epoch 61/100
2562/2562 [==============================] - 6s 2ms/sample - loss: 3.6816 - accuracy: 0.1772 - val_loss: 15.0521 - val_accuracy: 0.0764
Epoch 62/100
2562/2562 [==============================] - 5s 2ms/sample - loss: 3.6719 - accuracy: 0.1784 

In [ ]:
#mymodel.save('iq.h5')
plt.figure(figsize=(10,6))
import seaborn as sns
df=pd.DataFrame(mymodel.history.history)
plt.plot(df['accuracy'],label='accu')
plt.plot(df['val_accuracy'],label='val')
plt.legend()
plt.show()

In [ ]:
#df.plot()

from tensorflow.keras.preprocessing.sequence import pad_sequences
text=['missionaries wasted no time in establishing centres of learning in']
encoded=tokenizer.texts_to_sequences(text)[0]
print(encoded)
padded=pad_sequences([encoded],maxlen=25,truncating='pre') 
print(padded)
predind=mymodel.predict_classes([padded],verbose=0)[0]
print(tokenizer.index_word[predind])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generatetext(model,tokenizer,seq_len,text,num_gen_words):
    output_text=[]
    inputt=text
    for i in range(num_gen_words):
        encoded=tokenizer.texts_to_sequences(inputt)[0]
        print(encoded)
        padded=pad_sequences([encoded],maxlen=seq_len,truncating='pre')
        
        predind=model.predict_classes([padded],verbose=0)[0]
        
        word=tokenizer.index_word[predind]
        inputt.append(word)
        output_text.append(word)
    text.append(output_text)
    print(str(text))
text=['missionaries wasted no time in establishing centres of learning in']
generatetext(mymodel,tokenizer,seqln,text,8)